## **DATA PRE-PROCESSING**

### **Load Data**

In [1]:
from google.colab import files
data_to_load = files.upload()

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf

# data setup
import io
data = pd.read_csv(io.BytesIO(data_to_load['FULL_DATA.csv']),parse_dates=["DATE"])
data.head()


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213 entries, 0 to 212
Data columns (total 47 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   DATE        213 non-null    datetime64[ns]
 1   ALRCB       212 non-null    float64       
 2   CIC         211 non-null    float64       
 3   CPI         211 non-null    float64       
 4   CRUDEBRUSD  211 non-null    float64       
 5   EXSITC0     209 non-null    float64       
 6   EXSITC1     209 non-null    float64       
 7   EXSITC2     209 non-null    float64       
 8   EXSITC3     209 non-null    float64       
 9   EXSITC4     209 non-null    float64       
 10  EXSITC5     209 non-null    float64       
 11  EXSITC6     209 non-null    float64       
 12  EXSITC7     209 non-null    float64       
 13  EXSITC8     209 non-null    float64       
 14  EXSITC9     209 non-null    float64       
 15  EXTOT       211 non-null    float64       
 16  IMSITC0     209 non-null  

In [ ]:
# Import Necessary Libraries
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
from scipy.stats.mstats import zscore
import seaborn as sns
from pandas import Series
from matplotlib import pyplot
from statsmodels.tsa.stattools import adfuller
import itertools
import math
import random


In [ ]:
df=data.copy()

### **Missing Values**

In [ ]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [ ]:
mis_val= missing_values_table(df)
mis_val

Your selected dataframe has 47 columns.
There are 42 columns that have missing values.


,Missing Values,% of Total Values
RETRENCH,158,74.2
GDPGR,147,69.0
JOBVACAN,145,68.1
ACTIVEJOBS,145,68.1
IMPCON,9,4.2
IMSITC0,4,1.9
IMSITC9,4,1.9
IMSITC8,4,1.9
IMSITC7,4,1.9
IMSITC5,4,1.9


In [ ]:
# Delete columns containing either 10% or more than 10% NaN Values
perc = 10.0
min_count =  int(((100-perc)/100)*df.shape[0] + 1)
mod_df = df.dropna( axis=1, 
                thresh=min_count)

In [ ]:
mod_df.shape

(213, 43)

In [ ]:
# Add a Quarterly variables back to DataFrame
q = df['ACTIVEJOBS']
q1 = df['JOBVACAN']
q2 = df['RETRENCH']
q3 = df['GDPGR']
mod_df = mod_df.join(q)
mod_df = mod_df.join(q1)
mod_df = mod_df.join(q2)
mod_df = mod_df.join(q3)

In [ ]:
mod_df

,DATE,ALRCB,CIC,CPI,CRUDEBRUSD,EXSITC0,EXSITC1,EXSITC2,EXSITC3,EXSITC4,...,SERVTAX,CONSCRE,RUBBER,PALMOIL,OBOCC,IMPCON,ACTIVEJOBS,JOBVACAN,RETRENCH,GDPGR
0,2005-01-01,5.980000,78.8,86.5,48.20,788.861275,150.089998,1048.181046,5309.253984,1977.781692,...,-91.347676,16.586150,-1.202379,-25.209614,17.021932,13.770724,NaN,NaN,NaN,NaN
1,2005-02-01,5.950000,79.2,86.6,51.75,721.091699,105.104960,916.562834,4819.221845,1696.436111,...,124.245916,17.012425,0.496053,-30.095037,18.740244,13.311949,NaN,NaN,NaN,NaN
2,2005-03-01,5.900000,79.7,86.6,55.40,936.103843,134.578062,1209.154040,6103.839315,2247.228259,...,41.275279,17.425686,-3.872908,-28.635682,18.691811,7.266388,33740.0,15369.0,734.0,NaN
3,2005-04-01,5.970000,79.4,86.8,49.72,843.887467,157.084688,1185.318329,5898.350316,1999.962327,...,-4.860056,16.898292,-5.918167,-27.502528,19.450792,-2.279316,NaN,NaN,NaN,NaN
4,2005-05-01,5.940000,79.0,87.5,51.97,862.581569,128.087299,1092.293109,5420.482805,2308.730344,...,24.088199,17.701599,-4.258431,-24.546425,18.779487,10.795153,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,2022-05-01,3.678608,121.3,126.6,112.37,3303.244412,193.162587,2070.914794,20505.907160,9234.820975,...,17.392567,2.826704,2.300000,50.328084,5.843024,NaN,NaN,NaN,NaN,NaN
209,2022-06-01,3.791285,121.5,127.4,120.08,NaN,NaN,NaN,NaN,NaN,...,47.865238,4.793087,6.200000,59.404777,12.630753,NaN,NaN,NaN,NaN,8.9
210,2022-07-01,4.089581,119.6,127.9,108.92,NaN,NaN,NaN,NaN,NaN,...,33.317886,6.529381,2.100000,-1.586533,16.108114,NaN,NaN,NaN,NaN,NaN
211,2022-08-01,4.235996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,33.822375,6.041812,NaN,-8.474204,5.096721,NaN,NaN,NaN,NaN,NaN


### **Missing Values - MICE**

In [ ]:
#MICE
import numpy as np 
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

df_2 = mod_df.copy()
df_3 = df_2.drop(['DATE'],axis=1)

imputer=IterativeImputer(imputation_order='ascending',max_iter=10,random_state=42,n_nearest_features=5)
imputer
imputed_dataset = imputer.fit_transform(df_3)

df_3 = pd.DataFrame(imputed_dataset, columns = ['ALRCB','CIC','CPI','CRUDEBRUSD','EXSITC0','EXSITC1','EXSITC2','EXSITC3','EXSITC4','EXSITC5','EXSITC6','EXSITC7','EXSITC8','EXSITC9','EXTOT','IMSITC0','IMSITC1','IMSITC2','IMSITC3','IMSITC4','IMSITC5','IMSITC6','IMSITC7','IMSITC8','IMSITC9','IMTOT','IPIELEC','IPIMFG','IPIMIN','IPITOT','LGC','LIC','TOURIST','USDEXC','SOPC','SALETAX','SERVTAX','CONSCRE','RUBBER','PALMOIL','IMPCON','OBOCC','ACTIVEJOBS','JOBVACAN','RETRENCH','GDPGR'])

print(df_3)
print(type(df_3))

df_3.head()

        ALRCB         CIC         CPI  CRUDEBRUSD      EXSITC0     EXSITC1  \
0    5.980000   78.800000   86.500000   48.200000   788.861275  150.089998   
1    5.950000   79.200000   86.600000   51.750000   721.091699  105.104960   
2    5.900000   79.700000   86.600000   55.400000   936.103843  134.578062   
3    5.970000   79.400000   86.800000   49.720000   843.887467  157.084688   
4    5.940000   79.000000   87.500000   51.970000   862.581569  128.087299   
..        ...         ...         ...         ...          ...         ...   
208  3.678608  121.300000  126.600000  112.370000  3303.244412  193.162587   
209  3.791285  121.500000  127.400000  120.080000  4149.193522  243.789841   
210  4.089581  119.600000  127.900000  108.920000  3735.386856  185.309108   
211  4.235996  115.559564  120.860718   65.171021  3011.662801  311.029521   
212  4.677685  116.465598  121.644309   66.276225  3041.240168  327.911347   

         EXSITC2       EXSITC3      EXSITC4       EXSITC5  ... 

/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,


,ALRCB,CIC,CPI,CRUDEBRUSD,EXSITC0,EXSITC1,EXSITC2,EXSITC3,EXSITC4,EXSITC5,...,SERVTAX,CONSCRE,RUBBER,PALMOIL,IMPCON,OBOCC,ACTIVEJOBS,JOBVACAN,RETRENCH,GDPGR
0,5.98,78.8,86.5,48.20,788.861275,150.089998,1048.181046,5309.253984,1977.781692,2414.038394,...,-91.347676,16.586150,-1.202379,-25.209614,17.021932,13.770724,149065.517546,215095.939041,1004.993119,4.012176
1,5.95,79.2,86.6,51.75,721.091699,105.104960,916.562834,4819.221845,1696.436111,2444.808902,...,124.245916,17.012425,0.496053,-30.095037,18.740244,13.311949,139681.940043,214505.297435,816.938218,4.419227
2,5.90,79.7,86.6,55.40,936.103843,134.578062,1209.154040,6103.839315,2247.228259,2902.855983,...,41.275279,17.425686,-3.872908,-28.635682,18.691811,7.266388,33740.000000,15369.000000,734.000000,4.856417
3,5.97,79.4,86.8,49.72,843.887467,157.084688,1185.318329,5898.350316,1999.962327,2749.274618,...,-4.860056,16.898292,-5.918167,-27.502528,19.450792,-2.279316,145595.097427,241311.087931,1539.686381,4.913872
4,5.94,79.0,87.5,51.97,862.581569,128.087299,1092.293109,5420.482805,2308.730344,2476.513782,...,24.088199,17.701599,-4.258431,-24.546425,18.779487,10.795153,141417.183627,223816.575939,1447.589305,4.251609


In [ ]:
df_3.isnull().sum()

ALRCB         0
CIC           0
CPI           0
CRUDEBRUSD    0
EXSITC0       0
EXSITC1       0
EXSITC2       0
EXSITC3       0
EXSITC4       0
EXSITC5       0
EXSITC6       0
EXSITC7       0
EXSITC8       0
EXSITC9       0
EXTOT         0
IMSITC0       0
IMSITC1       0
IMSITC2       0
IMSITC3       0
IMSITC4       0
IMSITC5       0
IMSITC6       0
IMSITC7       0
IMSITC8       0
IMSITC9       0
IMTOT         0
IPIELEC       0
IPIMFG        0
IPIMIN        0
IPITOT        0
LGC           0
LIC           0
TOURIST       0
USDEXC        0
SOPC          0
SALETAX       0
SERVTAX       0
CONSCRE       0
RUBBER        0
PALMOIL       0
IMPCON        0
OBOCC         0
ACTIVEJOBS    0
JOBVACAN      0
RETRENCH      0
GDPGR         0
dtype: int64

In [ ]:
#Adding back DATE column

date = df['DATE']
df_3 = df_3.join(date)

df_3.head()

,ALRCB,CIC,CPI,CRUDEBRUSD,EXSITC0,EXSITC1,EXSITC2,EXSITC3,EXSITC4,EXSITC5,...,CONSCRE,RUBBER,PALMOIL,IMPCON,OBOCC,ACTIVEJOBS,JOBVACAN,RETRENCH,GDPGR,DATE
0,5.98,78.8,86.5,48.20,788.861275,150.089998,1048.181046,5309.253984,1977.781692,2414.038394,...,16.586150,-1.202379,-25.209614,17.021932,13.770724,149065.517546,215095.939041,1004.993119,4.012176,2005-01-01
1,5.95,79.2,86.6,51.75,721.091699,105.104960,916.562834,4819.221845,1696.436111,2444.808902,...,17.012425,0.496053,-30.095037,18.740244,13.311949,139681.940043,214505.297435,816.938218,4.419227,2005-02-01
2,5.90,79.7,86.6,55.40,936.103843,134.578062,1209.154040,6103.839315,2247.228259,2902.855983,...,17.425686,-3.872908,-28.635682,18.691811,7.266388,33740.000000,15369.000000,734.000000,4.856417,2005-03-01
3,5.97,79.4,86.8,49.72,843.887467,157.084688,1185.318329,5898.350316,1999.962327,2749.274618,...,16.898292,-5.918167,-27.502528,19.450792,-2.279316,145595.097427,241311.087931,1539.686381,4.913872,2005-04-01
4,5.94,79.0,87.5,51.97,862.581569,128.087299,1092.293109,5420.482805,2308.730344,2476.513782,...,17.701599,-4.258431,-24.546425,18.779487,10.795153,141417.183627,223816.575939,1447.589305,4.251609,2005-05-01


In [ ]:
df_3.head()

,ALRCB,CIC,CPI,CRUDEBRUSD,EXSITC0,EXSITC1,EXSITC2,EXSITC3,EXSITC4,EXSITC5,...,CONSCRE,RUBBER,PALMOIL,IMPCON,OBOCC,ACTIVEJOBS,JOBVACAN,RETRENCH,GDPGR,DATE
0,5.98,78.8,86.5,48.20,788.861275,150.089998,1048.181046,5309.253984,1977.781692,2414.038394,...,16.586150,-1.202379,-25.209614,17.021932,13.770724,149065.517546,215095.939041,1004.993119,4.012176,2005-01-01
1,5.95,79.2,86.6,51.75,721.091699,105.104960,916.562834,4819.221845,1696.436111,2444.808902,...,17.012425,0.496053,-30.095037,18.740244,13.311949,139681.940043,214505.297435,816.938218,4.419227,2005-02-01
2,5.90,79.7,86.6,55.40,936.103843,134.578062,1209.154040,6103.839315,2247.228259,2902.855983,...,17.425686,-3.872908,-28.635682,18.691811,7.266388,33740.000000,15369.000000,734.000000,4.856417,2005-03-01
3,5.97,79.4,86.8,49.72,843.887467,157.084688,1185.318329,5898.350316,1999.962327,2749.274618,...,16.898292,-5.918167,-27.502528,19.450792,-2.279316,145595.097427,241311.087931,1539.686381,4.913872,2005-04-01
4,5.94,79.0,87.5,51.97,862.581569,128.087299,1092.293109,5420.482805,2308.730344,2476.513782,...,17.701599,-4.258431,-24.546425,18.779487,10.795153,141417.183627,223816.575939,1447.589305,4.251609,2005-05-01


In [ ]:
from google.colab import files
df_3.to_csv('GDPGR_clean.csv') 
files.download('GDPGR_clean.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### **Transformation - Federal reserve NY**

In [ ]:
from google.colab import files
data_to_load = files.upload()

Saving GDPGR_clean.csv to GDPGR_clean.csv


In [ ]:
%matplotlib inline
import io
import types
import numpy as np
import pandas as pd
#import statsmodels.api as sm

import matplotlib.pyplot as plt
import seaborn as sns
import math

In [ ]:
def transform(column, transforms):
    transformation = transforms[column.name]
    # For quarterly data like GDP, we will compute
    # annualized percent changes
    mult = 4 if column.index.freqstr[0] == 'Q' else 1
    
    # 1 => No transformation
    if transformation == 1:
        pass
    # 2 => First difference
    elif transformation == 2:
        column = column.diff()
    # 3 => Second difference
    elif transformation == 3:
        column = column.diff().diff()
    # 4 => Log
    elif transformation == 4:
        column = np.log(column)
    # 5 => Log first difference, multiplied by 100
    #      (i.e. approximate percent change)
    elif transformation == 5:
        column = np.log(column).diff() * 100
    # 6 => Log second difference, multiplied by 100
    elif transformation == 6:
        column = np.log(column).diff().diff() * 100
    # 7 => Exact percent change, multiplied by 100
    #      with optional annualization
    elif transformation == 7:
        column = ((column / column.shift(1))**mult - 1.0) * 100
    # 8 => Log first difference, multiplied by 100
    #      (i.e. approximate percent change)
    #      with optional multiplier for annualization
    elif transformation == 8:
        column = np.log(column).diff() * 100 * mult
    # 9 => Log second difference, multiplied by 100
    #      with optional multiplier for annualization
    elif transformation == 9:
        column = np.log(column).diff().diff() * 100 * mult
    # 10 => Exact percent change, multiplied by 100
    #      with optional annualization
    elif transformation == 10:
        column = ((column / column.shift(1)) - 1.0) * 100
    elif transformation == 11:
        column = np.arcsinh(column).diff() * 100
    elif transformation == 12:
        column = np.arccosh(column).diff() * 100
    elif transformation == 13:
        column = np.arctanh(column).diff() * 100

      
    return column

In [ ]:
def load_transformation():
        
    
    # 1. Download data
    #orig_m = (pd.read_csv(f'{base_url}/monthly/{vintage}.csv').dropna(how='all'))
    orig_m = (pd.read_csv(io.BytesIO(data_to_load['GDPGR_clean.csv']),parse_dates=["DATE"]))
    
    # 2. Extract transformation information
    transform_m = orig_m.iloc[0, 1:]
    orig_m = orig_m.iloc[1:]

    # 3. Extract the date as an index
    orig_m.index = pd.PeriodIndex(orig_m.DATE.tolist(), freq='M')
    orig_m.drop('DATE', axis=1, inplace=True)

    # 4. Apply the transformations
    dta_m = orig_m.apply(transform, axis=0, transforms=transform_m)

    orig_m=orig_m
    dta_m=dta_m
    transform_m=transform_m

    df =pd.DataFrame(dta_m)

    #return types.SimpleNamespace(orig_m=orig_m, dta_m=dta_m, )    

    return df

df = load_transformation()
df
      

,ALRCB,CIC,CPI,CRUDEBRUSD,EXSITC0,EXSITC1,EXSITC2,EXSITC3,EXSITC4,EXSITC5,...,CONSCRE,RUBBER,PALMOIL,IMPCON,OBOCC,ACTIVEJOBS,JOBVACAN,RETRENCH,GDPGR,Unnamed: 47
2005-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.012176,NaN
2005-02,-0.496013,0.506290,0.115532,7.105116,-8.982409,-35.626410,-13.418091,-9.683886,-15.344619,1.266593,...,2.533110,149.517364,-17.701760,9.602007,-3.379075,-6.501794,-0.274972,-20.717238,4.419227,NaN
2005-03,-0.832118,0.629279,0.000000,6.814327,26.095991,24.717612,27.704550,23.630549,28.116789,17.172810,...,2.396097,-254.109670,4.967788,-0.258410,-60.210891,-142.068390,-263.598194,-10.705435,4.856417,NaN
2005-04,1.163063,-0.377092,0.230665,-10.815266,-10.370720,15.463698,-1.990960,-3.424526,-11.656922,-5.435797,...,-3.068073,-41.481822,4.035003,3.974763,-424.310825,146.214539,275.373443,74.082464,4.913872,NaN
2005-05,-0.496823,-0.505011,0.803164,4.425080,2.191055,-20.406791,-8.173208,-8.448781,14.356938,-10.448526,...,4.636498,32.267903,11.362740,-3.507468,463.638296,-2.911519,-7.526006,-6.167911,4.251609,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05,3.276362,0.827791,0.554439,6.043299,-14.554418,-24.775213,-43.449768,17.813611,10.289883,-15.356665,...,17.580457,-114.218676,-14.609481,104.318787,30.675806,1.018839,-6.032314,-19.977429,11.901268,NaN
2022-06,2.914369,0.164739,0.629904,6.635873,22.800883,23.277180,44.588293,-31.153294,-22.249010,2.888716,...,50.885769,95.385903,16.578355,76.520600,-21.485173,-1.828027,15.191283,50.376195,8.900000,NaN
2022-07,7.340536,-1.576088,0.391685,-9.754078,-10.506259,-27.427819,-5.768441,-6.130671,-2.828403,-4.168704,...,30.424210,-103.665473,-601.941664,24.258732,-31.501560,-33.118038,-14.677464,-4.942789,9.502942,NaN
2022-08,3.420255,-3.436549,-5.660808,-51.356098,-21.535903,51.785785,-25.306885,-9.571997,-21.572138,-36.237765,...,-7.664206,119.835014,-159.181322,-114.219977,8.014521,50.978954,-32.729233,-76.982756,5.064571,NaN


In [ ]:
df.isnull().sum()

ALRCB            1
CIC              1
CPI              1
CRUDEBRUSD       1
EXSITC0          1
EXSITC1          1
EXSITC2          1
EXSITC3          1
EXSITC4          1
EXSITC5          1
EXSITC6          1
EXSITC7          1
EXSITC8          1
EXSITC9          1
EXTOT            1
IMSITC0          1
IMSITC1          1
IMSITC2          1
IMSITC3          1
IMSITC4          1
IMSITC5          1
IMSITC6          1
IMSITC7          1
IMSITC8          1
IMSITC9          1
IMTOT            1
IPIELEC          1
IPIMFG           1
IPIMIN           1
IPITOT           1
LGC              1
LIC              1
TOURIST          1
USDEXC           1
SOPC             1
SALETAX          1
SERVTAX          1
CONSCRE          1
RUBBER           1
PALMOIL          1
IMPCON           1
OBOCC            1
ACTIVEJOBS       1
JOBVACAN         1
RETRENCH         1
GDPGR            0
Unnamed: 47    213
dtype: int64

In [ ]:
df_tra = df.copy()
df_tra.shape

(213, 47)

In [ ]:
# making new data frame with dropped NA values 
new_df = df_tra.dropna(axis = 0, how ='any') 
    
new_df

,ALRCB,CIC,CPI,CRUDEBRUSD,EXSITC0,EXSITC1,EXSITC2,EXSITC3,EXSITC4,EXSITC5,...,CONSCRE,RUBBER,PALMOIL,IMPCON,OBOCC,ACTIVEJOBS,JOBVACAN,RETRENCH,GDPGR,Unnamed: 47


In [ ]:
from google.colab import files
data_to_load = files.upload()

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf

# data setup
import io
data = pd.read_csv(io.BytesIO(data_to_load['gdp_yoy.csv']),parse_dates=["DATE"])
data.head()

In [ ]:
df= data.copy()

In [ ]:
# creating features and label variable
X = df[['ALRCB','CIC','CPI','CRUDEBRUSD','EXSITC0','EXSITC1','EXSITC2','EXSITC3','EXSITC4','EXSITC5','EXSITC6','EXSITC7','EXSITC8','EXSITC9','EXTOT','IMSITC0','IMSITC1','IMSITC2','IMSITC3','IMSITC4','IMSITC5','IMSITC6','IMSITC7','IMSITC8','IMSITC9','IMTOT','IPIELEC','IPIMFG','IPIMIN','IPITOT','LGC','LIC','TOURIST','USDEXC','SOPC','SALETAX','SERVTAX','CONSCRE','RUBBER','PALMOIL','IMPCON','OBOCC','RETRENCH','ACTIVEJOBS','JOBVACAN']]
y = df[["GDPGR"]]

In [ ]:
# Function to check stationarity

def adfuller_test(series, signif=0.05, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")

In [ ]:
# Data Prep

AI_Model_data = pd.concat([y, X], axis=1, sort=False)
#AI_Model_data = AI_Model_data.dropna()
# Check stationarity

# ADF Test on each column
for name, column in AI_Model_data.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

In [ ]:
#Save transformation
from google.colab import files
df.to_csv('gdp_arcsinh.csv') 
files.download('gdp_arcsinh.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
data_to_load = files.upload()

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf

# data setup
import io
data = pd.read_csv(io.BytesIO(data_to_load['gdp_arcsinh.csv']),parse_dates=["DATE"])
data.head()

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,2], name = data.iloc[:,2].name, line = dict(color = '#FF0000'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,3], name = data.iloc[:,3].name, line = dict(color = '#CD853F'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,4], name = data.iloc[:,4].name, line = dict(color = '#483D8B'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,5], name = data.iloc[:,5].name, line = dict(color = '#00008B'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,6], name = data.iloc[:,6].name, line = dict(color = '#8B4513'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,7], name = data.iloc[:,7].name, line = dict(color = '#7B68EE'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,8], name = data.iloc[:,8].name, line = dict(color = '#800080'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,9], name = data.iloc[:,9].name, line = dict(color = '#87CEEB'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,10], name = data.iloc[:,10].name, line = dict(color = '#FFDAB9'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,11], name = data.iloc[:,11].name, line = dict(color = '#32CD32'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,12], name = data.iloc[:,12].name, line = dict(color = '#D8BFD8'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,13], name = data.iloc[:,13].name, line = dict(color = '#DDA0DD'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,14], name = data.iloc[:,14].name, line = dict(color = '#6495ED'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,15], name = data.iloc[:,15].name, line = dict(color = '#FFA07A'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,16], name = data.iloc[:,16].name, line = dict(color = '#4682B4'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,17], name = data.iloc[:,17].name, line = dict(color = '#556B2F'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,18], name = data.iloc[:,18].name, line = dict(color = '#9ACD32'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,19], name = data.iloc[:,19].name, line = dict(color = '#B0C4DE'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,20], name = data.iloc[:,20].name, line = dict(color = '#9370DB'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,21], name = data.iloc[:,21].name, line = dict(color = '#DAA520'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,22], name = data.iloc[:,22].name, line = dict(color = '#0000CD'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,23], name = data.iloc[:,23].name, line = dict(color = '#008080'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,24], name = data.iloc[:,24].name, line = dict(color = '#B8860B'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,25], name = data.iloc[:,25].name, line = dict(color = '#FFD700'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,26], name = data.iloc[:,26].name, line = dict(color = '#800000'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,27], name = data.iloc[:,27].name, line = dict(color = '#FF8C00'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,28], name = data.iloc[:,28].name, line = dict(color = '#B0E0E6'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,29], name = data.iloc[:,29].name, line = dict(color = '#FF00FF'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,30], name = data.iloc[:,30].name, line = dict(color = '#F4A460'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,31], name = data.iloc[:,31].name, line = dict(color = '#FFFF00'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,32], name = data.iloc[:,32].name, line = dict(color = '#D2691E'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,33], name = data.iloc[:,33].name, line = dict(color = '#7FFFD4'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,34], name = data.iloc[:,34].name, line = dict(color = '#0000FF'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,35], name = data.iloc[:,35].name, line = dict(color = '#FFEFD5'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,36], name = data.iloc[:,36].name, line = dict(color = '#EEE8AA'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,37], name = data.iloc[:,37].name, line = dict(color = '#FFA500'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,38], name = data.iloc[:,38].name, line = dict(color = '#9400D3'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,39], name = data.iloc[:,39].name, line = dict(color = '#FFDEAD'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,40], name = data.iloc[:,40].name, line = dict(color = '#66CDAA'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,41], name = data.iloc[:,41].name, line = dict(color = '#8B008B'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,42], name = data.iloc[:,42].name, line = dict(color = '#D2B48C'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,43], name = data.iloc[:,43].name, line = dict(color = '#ADFF2F'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,44], name = data.iloc[:,44].name, line = dict(color = '#008000'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,45], name = data.iloc[:,45].name, line = dict(color = '#F5DEB3'), opacity = 0.8))
fig.add_trace(go.Scatter(x=data.DATE, y=data.iloc[:,46], name = data.iloc[:,46].name, line = dict(color = '#000000'), opacity = 1.0))